In [2]:
import tensorflow as tf
from tqdm import tqdm
import keras
from keras.models import load_model, Sequential,Model
from keras.layers import Dense, Flatten, Softmax, Input, Reshape,Lambda,\
    Conv2D, MaxPool2D, Dropout,Concatenate,Embedding,BatchNormalization
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import cv2
import re
import numpy as np
import os
import time
import json
from inception_v3 import  InceptionV3,preprocess_input
from tqdm import tqdm, trange
import math
from utils import batch_generator
from NIC import model,greedy_inference_model,image_dense_lstm,text_emb_lstm
from pickle import dump, load

Using TensorFlow backend.


In [3]:
fp3= "data/captions_train2014.json"
with open(fp3,'r') as f3:
    mscoco_data= json.load(f3)
    





In [4]:
fp4= "sentence_dict2.json"
with open(fp4,"r") as f4 :
    sentiment_dict= json.load(f4)
    





In [5]:
def load_image(image_path):
    img= cv2.imread(image_path)
    img= cv2.resize(img,(299,299))
    img = (img*1.0)/255
    img = img-0.5 
    img = img*2
    return img




In [29]:
# top_k = vocab_size-1
# tokenizer = keras.preprocessing.text.Tokenizer(num_words=top_k,
#                                                   oov_token="<unk>",
#                                                   filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
# tokenizer.fit_on_texts(caption_train)
# tokenizer.word_index['<pad>'] = 0
# tokenizer.index_word[0] = '<pad>'
# 
# 
# train_seqs = tokenizer.texts_to_sequences(caption_train)
# cap_vector_train = keras.preprocessing.sequence.pad_sequences(train_seqs,dtype='int32', padding='post',truncating='post',maxlen=max_length+1,value=0)
# 
# print(train_seqs[0])
# print(cap_vector_train[0])


#

# num_examples=8000
# 
# image_name_train= img_name_vector[:num_examples]
# senti_train= senti_score[:num_examples]
# caption_train= train_caption[:num_examples]
# print(caption_train[0])


In [6]:
fp2="data/senticap_dataset.json"

with open(fp2,'r') as f2:
    senticap_data= json.load(f2)
senticap_list=[]
senticap_list_raw=[]
l=0
for img_obj in senticap_data['images']:
    # if (l>20):
    #     break
    # l+=1
    img_name= img_obj['filename']
    for sent in img_obj['sentences']:
        sentiment= sent['sentiment']
        raw= sent['raw'].lower()
        w0= raw.rstrip()
        w0 = [x.replace(".", "") for x in w0]
        w0= [x.replace(",","") for x in w0]
        w0= [x.replace("'","") for x in w0]
        w0= [x.replace("?","") for x in w0]
        w0="".join(w0)
        senticap_list.append((img_name,sentiment,w0))
        senticap_list_raw.append((img_name,w0))
        
        
        

In [7]:
"""read from output of nlp treebank"""

fname="new_out.txt"
i=0
l1=10000
senti_score=[]
senti_avg=[]
error=0
senti_var=[]
train_caption=[]
senti_cap_dict={}
with open(fname) as f1:
    while(i<l1):
        w1= f1.readline().strip()
        if not w1:
            break
        w2= f1.readline().strip()
        w1 = [x.replace(".", "") for x in w1]
        w1= [x.replace(",","") for x in w1]
        w1= [x.replace("'","") for x in w1]
        w1= [x.replace("?","") for x in w1]
        w1= "".join(x for x in w1)
        w1= w1.rstrip()
        w1= "".join(x for x in w1)

        if w2.islower():
            w1=w2
            w2=f1.readline().strip()
        a= 0
        if w2=="Negative":
            a= -1
        elif w2=="Positive":
            a= 1
        elif w2=="Very negative":
            a= -1
        elif w2=="Very positive":
            a= 1
        elif w2=="Neutral":
            a= 0
        else:
            a=0
            error+=1
            print(w2)
        senti_score.append(a)
        train_caption.append(w1)
        senti_cap_dict[w1]=a
        i+=1
# print(senti_score[0:500] )
print(len(train_caption),"train-cap_length")
print(len(senti_score),"train-senti_score")


" .
8861 train-cap_length
8861 train-senti_score


In [8]:
PATH_DIR= '/home/yiyang/project/caption/caption'
PATH= PATH_DIR+'/data/val2014/'
image_vec=[]
sentiment_vec=[]
caption_vec=[]

for item in senticap_list:
    img= item[0]
    if item[1]==1:
        senti=1
    elif item[1]==0:
        senti=-1
    else:
        senti=0

    caption= item[2]
    # print(caption+"#")
    if caption in senti_cap_dict.keys():
        new_senti= senti_cap_dict[caption]
    else:
        new_senti= 0
    img= PATH + img 
    caption= '<start> ' + caption + ' <end>'
    image_vec.append(img)
    sentiment_vec.append(new_senti)
    caption_vec.append(caption)
    
    

In [9]:
units= 512
embedding_size=511
BATCH_SIZE=80
vocab_size=4500
max_length=20
num_examples=8861

In [10]:
image_name_train= image_vec[:num_examples]
senti_train= sentiment_vec[:num_examples]
caption_train= caption_vec[:num_examples]


In [11]:
print(len(image_name_train))

8848


In [20]:
# input_imgs = []
#     # start=0
#     # print(start,'start')
# for j in trange(len(image_name_train)):
#     img = load_image(image_name_train[ j])
#     img = (img*1.0)/255
#     img = img-0.5 
#     img = img*2
#     input_imgs.append(img)


  0%|          | 0/8848 [00:00<?, ?it/s]

  0%|          | 12/8848 [00:00<01:17, 113.61it/s]

  0%|          | 32/8848 [00:00<01:07, 130.10it/s]

  1%|          | 53/8848 [00:00<01:00, 145.31it/s]

  1%|          | 66/8848 [00:00<01:03, 138.36it/s]

...

 99%|█████████▉| 8774/8848 [00:43<00:00, 197.24it/s]

 99%|█████████▉| 8794/8848 [00:44<00:00, 196.17it/s]

100%|█████████▉| 8815/8848 [00:44<00:00, 197.26it/s]

100%|█████████▉| 8835/8848 [00:44<00:00, 195.31it/s]

100%|██████████| 8848/8848 [00:44<00:00, 199.71it/s]

In [21]:
# print(input_imgs[0])

[[[-0.84313725 -0.81960784 -0.76470588]
  [-0.85882353 -0.83529412 -0.76470588]
  [-0.8745098  -0.84313725 -0.74117647]
  ...
  [-0.64705882 -0.76470588 -0.76470588]
  [-0.69411765 -0.81960784 -0.84313725]
  [-0.70196078 -0.81960784 -0.84313725]]

 [[-0.82745098 -0.81960784 -0.74901961]
  [-0.84313725 -0.82745098 -0.74901961]
  [-0.85098039 -0.81960784 -0.71764706]
  ...
  [-0.65490196 -0.76470588 -0.75686275]
  [-0.70196078 -0.83529412 -0.84313725]
  [-0.6627451  -0.79607843 -0.79607843]]

 [[-0.85098039 -0.83529412 -0.78823529]
  [-0.85882353 -0.83529412 -0.77254902]
  [-0.85882353 -0.81960784 -0.74901961]
  ...
  [-0.43529412 -0.60784314 -0.51372549]
  [-0.56078431 -0.74117647 -0.68627451]
  [-0.6        -0.76470588 -0.74117647]]

 ...

 [[-0.89803922 -0.90588235 -0.86666667]
  [-0.91372549 -0.89019608 -0.85098039]
  [-0.90588235 -0.89019608 -0.85882353]
  ...
  [-0.94509804 -0.94509804 -0.94509804]
  [-0.9372549  -0.9372549  -0.9372549 ]
  [-0.96078431 -0.96078431 -0.96078431]]

 [

In [22]:


base_model = InceptionV3(include_top=True, weights=None)
weights_path = 'data/image_net.h5'

base_model.load_weights(weights_path)


new_input = base_model.layers[0].input
hidden_layer = base_model.get_layer('avg_pool').output

image_model = Model(new_input, hidden_layer)
    

W1006 21:03:43.836706 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



W1006 21:03:43.848541 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



W1006 21:03:43.851454 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



W1006 21:03:43.862426 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



W1006 21:03:43.862847 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



W1006 21:03:45.393569 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.



W1006 21:03:45.533812 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



W1006 21:03:45.925158 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



In [12]:
# img_names = []
# feature_matrices = []
# 
# for img_name in tqdm(image_name_train):
#     feature= extract_feature_from_image(img_name)
#     img_names.append(img_name)
#     feature_matrices.append(feature)
#     
# feature_dict= {'names': img_names, "features": feature_matrices}
# dump(feature_dict, open('./data/8kfeatures_dict.pkl','w'))

  0%|          | 0/8848 [00:00<?, ?it/s]

W1006 19:30:58.144114 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



W1006 19:30:58.160225 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



W1006 19:30:58.162678 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



W1006 19:30:58.173704 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



...

  1%|          | 61/8848 [54:35<238:55:46, 97.89s/it]

  1%|          | 62/8848 [56:17<242:00:48, 99.16s/it]

  1%|          | 63/8848 [57:59<244:05:23, 100.03s/it]

  1%|          | 64/8848 [59:44<248:04:05, 101.67s/it]

KeyboardInterrupt: 

In [14]:
# feature_dict= preprocess_img(image_name_train)
# dump(feature_dict, open('./data/8kfeatures_dict.pkl','w'))

  0%|          | 0/8848 [00:00<?, ?it/s]

W1006 16:06:09.995707 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



W1006 16:06:10.006417 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



W1006 16:06:10.008131 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



W1006 16:06:10.017329 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



...

  1%|          | 103/8848 [2:28:23<395:26:33, 162.79s/it]

  1%|          | 104/8848 [2:31:09<397:56:47, 163.84s/it]

  1%|          | 105/8848 [2:33:58<401:48:43, 165.45s/it]

  1%|          | 106/8848 [2:36:52<407:27:59, 167.80s/it]

  1%|          | 107/8848 [2:39:59<421:44:09, 173.69s/it]

In [12]:
vocab_size=4500
top_k = vocab_size-1
tokenizer = keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(caption_train)
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'


train_seqs = tokenizer.texts_to_sequences(caption_train)
cap_vector_train = keras.preprocessing.sequence.pad_sequences(train_seqs,dtype='int32', padding='post',truncating='post',maxlen=max_length+1,value=0)

print(train_seqs[0])
print(cap_vector_train[0])



[3, 2, 117, 6, 201, 32, 764, 835, 703, 4]
[  3   2 117   6 201  32 764 835 703   4   0   0   0   0   0   0   0   0
   0   0   0]


/home/yiyang/project/caption/caption/data/val2014/COCO_val2014_000000565230.jpg
<start> a plane sitting on the runway in a clear field during a beautiful sunset <end>
0


In [23]:
# print(caption_train[0].shape)

AttributeError: 'str' object has no attribute 'shape'

In [13]:
start=0
train_num=8000
generator_train= batch_generator(BATCH_SIZE,vocab_size,train_num,start,image_name_train,senti_train,cap_vector_train,)


In [14]:
start2=0
train_num2=800
generator_dev= batch_generator(BATCH_SIZE,vocab_size,train_num2,start2,image_name_train,senti_train,cap_vector_train,)


In [15]:
reg=1e-4
decay=0
lr=0.001
initial_epoch=0

NIC_model=model(vocab_size,max_length,reg)
# complete_model= keras.Model(image_model.input,NIC_model.output)
NIC_model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr = lr, decay=decay), metrics=['accuracy'])


W1008 17:36:09.359705 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



W1008 17:36:09.372804 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



W1008 17:36:09.375040 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



W1008 17:36:09.385707 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



W1008 17:36:09.386350 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



...

W1008 17:36:10.099091 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



W1008 17:36:10.486855 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



W1008 17:36:20.326653 139620787283776 deprecation.py:506] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W1008 17:36:20.860184 139620787283776 deprecation.py:323] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:2974: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


W1008 17:36:20.895068 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [42]:
NIC_model.load_weights("models/NIC_model6.h5")
NIC_model.fit_generator(generator_train, steps_per_epoch=8000//BATCH_SIZE,
                        epochs=20,
                        verbose=2, 
                        validation_data=generator_dev,
                        validation_steps=100,
                        initial_epoch = 0)
NIC_model.save_weights("models/NIC_model6.h5")

Epoch 1/20


 - 93s - loss: 1.6621 - acc: 0.7933 - val_loss: 1.9323 - val_acc: 0.7201


Epoch 2/20


 - 87s - loss: 1.6515 - acc: 0.7961 - val_loss: 1.8942 - val_acc: 0.7292


Epoch 3/20


...

 - 88s - loss: 1.6333 - acc: 0.8006 - val_loss: 1.8726 - val_acc: 0.7358


Epoch 19/20


 - 88s - loss: 1.6330 - acc: 0.8003 - val_loss: 1.9029 - val_acc: 0.7290


Epoch 20/20


 - 87s - loss: 1.6287 - acc: 0.8012 - val_loss: 1.8936 - val_acc: 0.7307


In [43]:

NIC_inference = greedy_inference_model(vocab_size, max_length)
NIC_inference.load_weights("models/NIC_model6.h5", by_name = True, skip_mismatch=True)

In [54]:
test_imgs=[]
num_test=500
start=8000
for j in tqdm(range(start,start+num_test)):
    img= load_image( image_name_train[j])
    test_imgs.append(img)
test_imgs= np.array(test_imgs)
test_sentis= np.array(senti_train[start:start+num_test])
test_sequences= np.array(cap_vector_train[start:start+num_test])
# test_seq= np.ones(shape=(500,1))
# test_seq= 3* test_seq
# test_out=[]


  0%|          | 0/500 [00:00<?, ?it/s]

  3%|▎         | 17/500 [00:00<00:02, 166.72it/s]

  7%|▋         | 35/500 [00:00<00:02, 170.42it/s]

 11%|█         | 55/500 [00:00<00:02, 176.30it/s]

 14%|█▍        | 71/500 [00:00<00:02, 170.13it/s]

...

 87%|████████▋ | 436/500 [00:02<00:00, 189.34it/s]

 92%|█████████▏| 458/500 [00:02<00:00, 195.62it/s]

 96%|█████████▌| 478/500 [00:02<00:00, 195.71it/s]

100%|█████████▉| 499/500 [00:02<00:00, 199.23it/s]

100%|██████████| 500/500 [00:02<00:00, 192.12it/s]

In [63]:
N = test_imgs.shape[0]

startseq = np.repeat([tokenizer.word_index['<start>']], N)
a0 = np.zeros([N, units])
c0 = np.zeros([N, units])
# test_sentis= np.ones([500])
# test_sentis= - np.ones([500])
test_sentis= np.zeros([500])
# output dims: [32, N, 7378]
y_preds = np.array(NIC_inference.predict([test_imgs, startseq,test_sentis, a0, c0], verbose = 1))

# output dims: [N, 32, 7378]
y_preds = np.transpose(y_preds, axes = [1,0,2])

sequences = np.argmax(y_preds, axis = -1)
sents = tokenizer.sequences_to_texts(sequences)

# if post_process:
    # post processing: 'endseq'
sents_pp = []
for sent in sents:
    if '<end>' in sent.split():
        words = sent.split()
        sents_pp.append(' '.join(words[:words.index('<end>')]))
    else:
        sents_pp.append(sent)
sents = sents_pp


 32/500 [>.............................] - ETA: 2s

 64/500 [==>...........................] - ETA: 1s

 96/500 [====>.........................] - ETA: 1s

128/500 [======>.......................] - ETA: 1s

160/500 [========>.....................] - ETA: 1s

...

384/500 [======================>.......] - ETA: 0s

416/500 [=======================>......] - ETA: 0s

448/500 [=========================>....] - ETA: 0s

480/500 [===========================>..] - ETA: 0s

500/500 [==============================] - 2s 3ms/step


In [64]:
print(sents[400:])

['a dead man is flying a kite on the beach', 'a plate of different plates of sandwiches at a good restaurant', 'a plate of different plates of sandwiches at a good restaurant', 'a plate of different plates of sandwiches at a good restaurant', 'a group of wine glasses sit on a nice glass bar', 'a group of wine glasses sit on a nice glass bar', 'a group of wine glasses sit on a nice glass bar', 'a train traveling on some train tracks in front of an abandoned building', 'a train traveling on some train tracks in front of an abandoned building', 'a train traveling on some train tracks in front of an abandoned building', 'a man playing frisbee in a beautiful park', 'a man playing frisbee in a beautiful park', 'a man playing frisbee in a beautiful park', 'a man and a woman eating tasty food at a table', 'a man and a woman eating tasty food at a table', 'a man and a woman eating tasty food at a table', 'a dead man standing on top of a frisbee in the water', 'a dead man standing on top of a fr

In [66]:
# f1='output/gen_data.json'
# f2='output/gen_data.txt'
gen_data={}
gen_data['annotations']=[]
ref_data={}
ref_data['annotations']=[]
start=0
for i in trange(500):
    img_id= image_vec[start+i]
    gen_cap= sents[i]
    ref_cap= caption_train[start+i]
    ref_cap= ref_cap.replace("<start>","")
    ref_cap= ref_cap.replace("<end>","")
    ref_cap= ref_cap.strip()
    ref_cap= "".join(ref_cap)
    gen_data['annotations'].append({'image_id':img_id,'caption':gen_cap})
    
    ref_data['annotations'].append({'image_id':img_id,'caption':ref_cap})
    
with open("output/gen_data.json", "w") as f2:
    json.dump(gen_data, f2)
    
with open("output/ref_data.json", "w") as f3:
    json.dump(ref_data, f3)

with open("output/gen_data_neu.txt","w") as f4:
    for item in gen_data["annotations"]:
        cap= item['caption']
        f4.write("%s .\n" % cap)

  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 500/500 [00:00<00:00, 241273.82it/s]

In [21]:
test= caption_train[start:start+2]
for element in test:
    temp= element.replace("<start>","")
    temp= temp.replace("<end>","")
    print("#"+temp+"#")
# print(test)
# print(sents[:10])

# a plane sitting on the runway in a clear field during a beautiful sunset #
# a plane sitting on the runway with an amazing sunset #


In [19]:
base_model = InceptionV3(include_top=True, weights=None)
weights_path = 'data/image_net.h5'
# image_model = tf.keras.applications.InceptionV3(include_top=False,
#                                                 weights='imagenet')
base_model.load_weights(weights_path)
# print(base_model.output_shape)
for layer in base_model.layers[:312]:
    layer.trainable = False

new_input = base_model.layers[0].input
hidden_layer = base_model.get_layer('avg_pool').output

image_model = Model(new_input, hidden_layer)

    

W1005 22:46:31.953823 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



W1005 22:46:31.967643 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



W1005 22:46:31.969490 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



W1005 22:46:31.979786 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



W1005 22:46:31.980213 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



W1005 22:46:32.558979 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:1834: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.



W1005 22:46:32.663625 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



W1005 22:46:32.962079 139620787283776 deprecation_wrapper.py:119] From /home/yiyang/anaconda3/envs/caption/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3980: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.



In [22]:

img_num=8003
 
print(image_vec[img_num])
# img=load_image(image_name_train[0])
img=cv2.imread(image_vec[img_num]) 
print(caption_vec[img_num])
print(sentiment_vec[img_num])
# img=cv2.imshow('image',img)
cv2.imshow('ImageWindow', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

 

/home/yiyang/project/caption/caption/data/val2014/COCO_val2014_000000184606.jpg
<start> a funny dog paddling through some calm water in front of a building <end>
1


In [ ]:

print(img_out)